In [1]:
import pickle
from pathlib import Path

import pandas as pd
import numpy as np
from pulearn.elkanoto import ElkanotoPuClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import ParameterGrid, GridSearchCV
from sklearn.decomposition import PCA

from tqdm import tqdm
import pul_config
import importlib
importlib.reload(pul_config)

import utils
importlib.reload(utils)
from utils import calc_predictions, build_proba_df, build_ranked_lists, calc_scores, load_grid_searches, score_df_from_grid_searches, train, evaluate

# CHANGE HERE
input_dirs = [Path('1_outputs/standard/'), Path('1_outputs/small_DS/'), Path('1_outputs/large/'), Path('1_outputs/large_small_DS/')]
output_dirs = [Path('2_outputs/standard/'), Path('2_outputs/small_DS/'), Path('2_outputs/large/'), Path('2_outputs/large_small_DS/')]
method = 'OneClassSVM'
pul_cfg = pul_config.OneClassSVMConfig

remote_method = "OneClassSVM" # name of the output in S3
# -----

local_method = method
MODEL_NAMES = ('RotatE', 'TransE', 'CompGCN')
output_dirs = [output_dir / method for output_dir in output_dirs]

## Training

In [ ]:
N_JOBS = 20
CV = 5
rerun=False

for input_dir, output_dir in tqdm(zip(input_dirs, output_dirs), total=len(input_dirs)):
    train(input_dir=input_dir, output_dir=output_dir, model_names=MODEL_NAMES, cv=CV, n_jobs=N_JOBS, pul_cfg=pul_cfg, rerun=rerun)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

 40%|████      | 4/10 [00:00<00:00, 34.18it/s]

2_outputs/standard/OneClassSVM/RotatE/RotatE_0.pkl exists. Skipping...
2_outputs/standard/OneClassSVM/RotatE/RotatE_1.pkl exists. Skipping...
2_outputs/standard/OneClassSVM/RotatE/RotatE_2.pkl exists. Skipping...
2_outputs/standard/OneClassSVM/RotatE/RotatE_3.pkl exists. Skipping...
2_outputs/standard/OneClassSVM/RotatE/RotatE_4.pkl exists. Skipping...
2_outputs/standard/OneClassSVM/RotatE/RotatE_5.pkl exists. Skipping...
2_outputs/standard/OneClassSVM/RotatE/RotatE_6.pkl exists. Skipping...




 80%|████████  | 8/10 [00:00<00:00, 29.90it/s]

2_outputs/standard/OneClassSVM/RotatE/RotatE_7.pkl exists. Skipping...


100%|██████████| 10/10 [00:00<00:00, 30.72it/s]

 33%|███▎      | 1/3 [00:00<00:00,  2.86it/s]

...



  0%|          | 0/10 [00:00<?, ?it/s]

2_outputs/standard/OneClassSVM/CompGCN/CompGCN_0.pkl exists. Skipping...
2_outputs/standard/OneClassSVM/CompGCN/CompGCN_1.pkl exists. Skipping...
2_outputs/standard/OneClassSVM/CompGCN/CompGCN_2.pkl exists. Skipping...
2_outputs/standard/OneClassSVM/CompGCN/CompGCN_3.pkl exists. Skipping...




 60%|██████    | 6/10 [00:00<00:00, 54.19it/s]

2_outputs/standard/OneClassSVM/CompGCN/CompGCN_4.pkl exists. Skipping...
2_outputs/standard/OneClassSVM/CompGCN/CompGCN_5.pkl exists. Skipping...
2_outputs/standard/OneClassSVM/CompGCN/CompGCN_6.pkl exists. Skipping...
2_outputs/standard/OneClassSVM/CompGCN/CompGCN_7.pkl exists. Skipping...
2_outputs/standard/OneClassSVM/CompGCN/CompGCN_8.pkl exists. Skipping...




 60%|██████    | 6/10 [00:19<00:00, 54.19it/s]

## Evaluation

In [ ]:
grid_searches_list = [
    load_grid_searches(output_dir=o, model_names=MODEL_NAMES) for o in output_dirs
]

In [ ]:
score_dfs = [score_df_from_grid_searches(gs, i) for gs, i in zip(grid_searches_list, input_dirs)]
for output_dir, score_df in zip(output_dirs, score_dfs):
    score_df.to_csv(output_dir / 'score_df.csv')

In [ ]:
score_dfs[0].groupby('model_name').mean()

In [ ]:
import seaborn as sns

sns.boxplot(score_dfs[0], x='model_name', y='score10_test').set(title='Hits@100 and hits@10\n(large positive set)')
sns.boxplot(score_dfs[0], x='model_name', y='score100_test').set_ylabel('score')

In [ ]:
import seaborn as sns

sns.boxplot(score_dfs[1], x='model_name', y='score10_test').set(title='Hits@100 and hits@10\n(small positive set)')
sns.boxplot(score_dfs[1], x='model_name', y='score100_test').set_ylabel('score')

In [ ]:
import seaborn as sns

sns.boxplot(score_dfs[2], x='model_name', y='score10_test').set(title='Hits@100 and hits@10\n(large positive set, large embeddings)')
sns.boxplot(score_dfs[2], x='model_name', y='score100_test').set_ylabel('score')

In [ ]:
import seaborn as sns

sns.boxplot(score_dfs[3], x='model_name', y='score10_test').set(title='Hits@100 and hits@10\n(large positive set, small embeddings)')
sns.boxplot(score_dfs[3], x='model_name', y='score100_test').set_ylabel('score')

In [ ]:
score_dfs = []
for output_dir in output_dirs:
    score_dfs.append(pd.read_csv(output_dir / 'score_df.csv', index_col=0))
plot_dfs = []
for ds, score_df in zip(('large', 'small', 'large_embeddings', 'large_embeddings_small'), score_dfs):
    plot_df = score_df[['model_name', 'score10_test', 'score100_test']].copy()
    plot_df.columns = ['model_name', 'hits@10', 'hits@100']
    plot_df['dataset'] = ds
    plot_dfs.append(plot_df)

plot_df = pd.concat(plot_dfs)
plot_df = plot_df.melt(id_vars=['model_name', 'dataset'])
plot_dfs = {k:v for k, v in plot_df.groupby('model_name')}

In [ ]:
y_lim = (0, 105)
y_label = 'Score'

import seaborn as sns
import matplotlib.pyplot as plt

model_name = 'RotatE'
plot_df = plot_dfs[model_name]

plot_df['Hits'] = plot_df['variable']
plot = sns.barplot(plot_df, x='dataset', y='value', hue='Hits', order=['small', 'large', 'large_embeddings_small', 'large_embeddings'])
sns.move_legend(plot, 'upper left')
plot.set(xlabel='positive dataset', ylabel=y_label)
plot.set_title(f'{model_name} - {method}')
plot.set_ylim(y_lim)
_ = plot.set_xticklabels(labels=['small', 'large', 'large_embeddings_small', 'large_embeddings'], rotation=90)

In [ ]:
y_lim = (0, 105)
y_label = 'Score'

import seaborn as sns
import matplotlib.pyplot as plt

model_name = 'TransE'
plot_df = plot_dfs[model_name]

plot_df['Hits'] = plot_df['variable']
plot = sns.barplot(plot_df, x='dataset', y='value', hue='Hits', order=['small', 'large', 'large_embeddings_small', 'large_embeddings'])
sns.move_legend(plot, 'upper left')
plot.set(xlabel='positive dataset', ylabel=y_label)
plot.set_title(f'{model_name} - {method}')
plot.set_ylim(y_lim)
_ = plot.set_xticklabels(labels=['small', 'large', 'large_embeddings_small', 'large_embeddings'], rotation=90)

In [ ]:
y_lim = (0, 105)
y_label = 'Score'

import seaborn as sns
import matplotlib.pyplot as plt

model_name = 'CompGCN'
plot_df = plot_dfs[model_name]

plot_df['Hits'] = plot_df['variable']
plot = sns.barplot(plot_df, x='dataset', y='value', hue='Hits', order=['small', 'large', 'large_embeddings_small', 'large_embeddings'])
sns.move_legend(plot, 'upper left')
plot.set(xlabel='positive dataset', ylabel=y_label)
plot.set_title(f'{model_name} - {method}')
plot.set_ylim(y_lim)
_ = plot.set_xticklabels(labels=['small', 'large', 'large_embeddings_small', 'large_embeddings'], rotation=90)

In [ ]:
proba_dfs = [build_proba_df(gs, i, mns=MODEL_NAMES) for gs, i in zip(grid_searches_list, input_dirs)]
for output_dir, proba_df in zip(output_dirs, proba_dfs):
    proba_df.to_csv(output_dir / 'proba_df.csv')

In [ ]:
proba_df = proba_dfs[0]
proba_df.query('test').sort_values('RotatE_0', ascending=False)

In [ ]:
for proba_df, score_df, od in zip(proba_dfs, score_dfs, output_dirs):
    proba_df.to_csv(od / 'proba_df.csv')
    proba_df.query('test').to_csv(od / 'proba_df_test.csv')
    score_df.to_csv(od / 'score_df.csv')
    
    for mn in MODEL_NAMES:
        score_cols = [c for c in proba_df.columns if c.startswith(f'{mn}_')]
        tmp_df = proba_df[['node_id', 'gene_name', 'irr', 'train', 'val', 'test', *score_cols]]
        tmp_df.to_csv(od / f'proba_df_{mn}.csv')
        tmp_df_test = tmp_df.query('test')
        tmp_df_test.to_csv(od / f'proba_df_test_{mn}.csv')
        
        genes_100 = build_ranked_lists(tmp_df, prefix=mn)
        genes_100.to_csv(od / f'genes_100_{mn}.csv', index=False)
        genes_100_test = build_ranked_lists(tmp_df_test, prefix=mn)
        genes_100_test.to_csv(od / f'genes_100_test_{mn}.csv', index=False)
        
        ds = 'test'
        score_df_tmp = score_df.query('model_name == @mn')
        score_df_tmp = pd.concat([score_df_tmp['m_name'], score_df_tmp.filter(regex=f'.*_{ds}', axis=1)], axis=1).set_index('m_name')
        score_df_tmp.columns = [c.replace(f'_{ds}', '').replace('score', 'hits@') for c in score_df_tmp.columns]
        score_df_tmp = score_df_tmp.T
        score_df_tmp['mean'] = score_df_tmp.mean(axis=1)
        score_df_tmp.to_csv(od / f'scores_{ds}_{mn}.csv', index_label='score')

        ds = 'all'
        score_df_tmp = score_df.query('model_name == @mn')
        score_df_tmp = pd.concat([score_df_tmp['m_name'], score_df_tmp.filter(regex=f'.*_{ds}', axis=1)], axis=1).set_index('m_name')
        score_df_tmp.columns = [c.replace(f'_{ds}', '').replace('score', 'hits@') for c in score_df_tmp.columns]
        score_df_tmp = score_df_tmp.T
        score_df_tmp['mean'] = score_df_tmp.mean(axis=1)
        score_df_tmp.to_csv(od / f'scores_{mn}.csv', index_label='score')

In [ ]:
s3_prefix = "s3://nnedl-core-prd-eu-central-1-curated/compbiosandbox/Projects/NNRCO_KG/Sprint_3_3_IR_Paper/Result_lists"

for model_name in MODEL_NAMES:
    method_name = f"{remote_method}_{model_name}"

    graph = "Hetionet"
    ts = "LargeTS"
    subset = "WholeSet"
    !aws s3 cp './2_outputs/standard/{local_method}/proba_df_{model_name}.csv' {s3_prefix}/{graph}/{ts}/{method_name}/{subset}/all_results.csv
    !aws s3 cp './2_outputs/standard/{local_method}/genes_100_{model_name}.csv' {s3_prefix}/{graph}/{ts}/{method_name}/{subset}/gene_100.csv
    !aws s3 cp './2_outputs/standard/{local_method}/scores_{model_name}.csv' {s3_prefix}/{graph}/{ts}/{method_name}/{subset}/scores.csv
    subset = "TestSet"
    !aws s3 cp './2_outputs/standard/{local_method}/proba_df_test_{model_name}.csv' {s3_prefix}/{graph}/{ts}/{method_name}/{subset}/all_results.csv
    !aws s3 cp './2_outputs/standard/{local_method}/genes_100_test_{model_name}.csv' {s3_prefix}/{graph}/{ts}/{method_name}/{subset}/gene_100.csv
    !aws s3 cp './2_outputs/standard/{local_method}/scores_test_{model_name}.csv' {s3_prefix}/{graph}/{ts}/{method_name}/{subset}/scores.csv

    ts = "SmallTS"
    subset = "WholeSet"
    !aws s3 cp './2_outputs/small_DS/{local_method}/proba_df_{model_name}.csv' {s3_prefix}/{graph}/{ts}/{method_name}/{subset}/all_results.csv
    !aws s3 cp './2_outputs/small_DS/{local_method}/genes_100_{model_name}.csv' {s3_prefix}/{graph}/{ts}/{method_name}/{subset}/gene_100.csv
    !aws s3 cp './2_outputs/small_DS/{local_method}/scores_{model_name}.csv' {s3_prefix}/{graph}/{ts}/{method_name}/{subset}/scores.csv
    subset = "TestSet"
    !aws s3 cp './2_outputs/small_DS/{local_method}/proba_df_test_{model_name}.csv' {s3_prefix}/{graph}/{ts}/{method_name}/{subset}/all_results.csv
    !aws s3 cp './2_outputs/small_DS/{local_method}/genes_100_test_{model_name}.csv' {s3_prefix}/{graph}/{ts}/{method_name}/{subset}/gene_100.csv
    !aws s3 cp './2_outputs/small_DS/{local_method}/scores_test_{model_name}.csv' {s3_prefix}/{graph}/{ts}/{method_name}/{subset}/scores.csv

    # Embeddings based on large DS
    graph = "enrichedHetionet"

    ts = "LargeTS"
    subset = "WholeSet"
    !aws s3 cp './2_outputs/large/{local_method}/proba_df_{model_name}.csv' {s3_prefix}/{graph}/{ts}/{method_name}/{subset}/all_results.csv
    !aws s3 cp './2_outputs/large/{local_method}/genes_100_{model_name}.csv' {s3_prefix}/{graph}/{ts}/{method_name}/{subset}/gene_100.csv
    !aws s3 cp './2_outputs/large/{local_method}/scores_{model_name}.csv' {s3_prefix}/{graph}/{ts}/{method_name}/{subset}/scores.csv
    subset = "TestSet"
    !aws s3 cp './2_outputs/large/{local_method}/proba_df_test_{model_name}.csv' {s3_prefix}/{graph}/{ts}/{method_name}/{subset}/all_results.csv
    !aws s3 cp './2_outputs/large/{local_method}/genes_100_test_{model_name}.csv' {s3_prefix}/{graph}/{ts}/{method_name}/{subset}/gene_100.csv
    !aws s3 cp './2_outputs/large/{local_method}/scores_test_{model_name}.csv' {s3_prefix}/{graph}/{ts}/{method_name}/{subset}/scores.csv

    ts = "SmallTS"
    subset = "WholeSet"
    !aws s3 cp './2_outputs/large_small_DS/{local_method}/proba_df_{model_name}.csv' {s3_prefix}/{graph}/{ts}/{method_name}/{subset}/all_results.csv
    !aws s3 cp './2_outputs/large_small_DS/{local_method}/genes_100_{model_name}.csv' {s3_prefix}/{graph}/{ts}/{method_name}/{subset}/gene_100.csv
    !aws s3 cp './2_outputs/large_small_DS/{local_method}/scores_{model_name}.csv' {s3_prefix}/{graph}/{ts}/{method_name}/{subset}/scores.csv
    subset = "TestSet"
    !aws s3 cp './2_outputs/large_small_DS/{local_method}/proba_df_test_{model_name}.csv' {s3_prefix}/{graph}/{ts}/{method_name}/{subset}/all_results.csv
    !aws s3 cp './2_outputs/large_small_DS/{local_method}/genes_100_test_{model_name}.csv' {s3_prefix}/{graph}/{ts}/{method_name}/{subset}/gene_100.csv
    !aws s3 cp './2_outputs/large_small_DS/{local_method}/scores_test_{model_name}.csv' {s3_prefix}/{graph}/{ts}/{method_name}/{subset}/scores.csv